In [1]:
import pandas as pd
import re
import csv
import numpy as np

### Outlier decetion of internal data

In [2]:
# Converte json document to csv.
pdObj = pd.read_json('../data/curated/domain.json', orient='index')
pdObj.to_csv('../data/curated/streaming_test.csv', index= True)

In [3]:
# Get the cost
df = pd.read_csv('../data/curated/streaming_test.csv')
df['cost'] = df['cost_text']
for i in range(len(df)):
    df['cost'].iloc[i] = re.findall(r'\d+\.?\d*',df['cost_text'].iloc[i])
df = df.rename({'Unnamed: 0':'url', 'suburb_ower':'suburb_owner'},axis=1)

/Users/victor/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
# Delete missing values if cost is 0
delete_list = []
df_new = df
for i in range(len(df)):
    if len(df['cost'].iloc[i]) == 0:
        delete_list.append(i)
df_new = df_new.drop(df_new.index[delete_list])

In [5]:
# Transform the data type of cost into float.
for i in range(len(df_new)):
    df_new['cost'].iloc[i] = float(df_new['cost'].iloc[i][0])
    
df_new['cost'] = df_new['cost'].astype('float')

In [6]:
# Delete the extreme values: delete Highest 20% and lowest 20% cost.
delete_list = []
k1 = 0.8
k2 = 0.2
df_new2 = df_new
for i in range(len(df_new2)):
    if df_new2['cost'].iloc[i] < df_new2['cost'].mean()*(1-k1) or df_new2['cost'].iloc[i] > df_new2['cost'].mean()*(1+k2):
        delete_list.append(i)
df_new2 = df_new2.drop(df_new2.index[delete_list])
df_new2.describe()

,postcode,cost
count,13231.000000,13231.000000
mean,3203.883531,474.439385
std,251.114949,137.840632
min,3000.000000,160.000000
25%,3044.000000,380.000000
50%,3125.000000,450.000000
75%,3204.000000,550.000000
max,3996.000000,931.000000


In [7]:
# Divide rooms into 'Bed', 'Bath' and 'Parking'
df_new2['Beds'] = df_new2['rooms']
df_new2['Bath'] = df_new2['rooms']
df_new2['Parking'] = df_new2['rooms']

for i in range(len(df_new2)):
    try:
        df_new2['Beds'].iloc[i] = eval(df_new2['rooms'].iloc[i])[0][0]
    except:
        df_new2['Beds'].iloc[i] = 0
    try:
        df_new2['Bath'].iloc[i] = eval(df_new2['rooms'].iloc[i])[1][0]
    except:
        df_new2['Bath'].iloc[i] = 0
    try:
        df_new2['Parking'].iloc[i] = eval(df_new2['rooms'].iloc[i])[2][0]
    except:
        df_new2['Parking'].iloc[i] = 0

/Users/victor/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
domain = df_new2

In [9]:
domain.head()

,url,name,cost_text,type,school_title,school_distance,suburb_sold,suburb_avg_day,suburb_population,suburb_age,...,suburb_family,suburb_single,coordinates,rooms,desc,postcode,cost,Beds,Bath,Parking
0,https://www.domain.com.au/1202-620-collins-str...,1202/620 Collins Street Melbourne VIC 3000,$500 per week,Apartment / Unit / Flat,University High School,2.4 km away,596,147,"47,279",20 to 39,...,24%,76%,"[-37.8186626, 144.9543471]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Ender Gok,3000,500.0,2 Beds,1 Bath,1 Parking
1,https://www.domain.com.au/1701-620-collins-str...,1701/620 Collins Street Melbourne VIC 3000,$650 per week,Apartment / Unit / Flat,University High School,2.4 km away,596,147,"47,279",20 to 39,...,24%,76%,"[-37.8186626, 144.9543471]","[['2 Beds'], ['2 Baths'], ['1 Parking']]",Ender Gok,3000,650.0,2 Beds,2 Baths,1 Parking
2,https://www.domain.com.au/5707b-639-little-lon...,5707B/639 Little Lonsdale Street Melbourne VIC...,$570,Apartment / Unit / Flat,University High School,1.9 km away,387,120,"47,279",20 to 39,...,24%,76%,"[-37.8141942, 144.9521662]","[['1 Bed'], ['1 Bath'], []]",Kim Te,3000,570.0,1 Bed,1 Bath,0
3,https://www.domain.com.au/1003-225-elizabeth-s...,1003/225 Elizabeth Street Melbourne VIC 3000,$575.00 per week,Apartment / Unit / Flat,Carlton Gardens Primary School,1.4 km away,596,147,"47,279",20 to 39,...,24%,76%,"[-37.8132308, 144.9623173]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Barbara Baltas,3000,575.0,2 Beds,1 Bath,1 Parking
4,https://www.domain.com.au/1202-31-spring-stree...,1202/31 Spring Street Melbourne VIC 3000,$570.00,Apartment / Unit / Flat,Carlton Gardens Primary School,1.4 km away,596,147,"47,279",20 to 39,...,24%,76%,"[-37.8145047, 144.9740695]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Bo Natoli,3000,570.0,2 Beds,1 Bath,1 Parking


Some suburb data is missing in domian, this is probably because there are not enough property transactions in this postcide suburb on domian to be counted, so we try to fill in the data with neighbouring suburb, or delete them if there are no near suburb or only a few data is missing

In [10]:
domain[domain['suburb_population'].isnull()]['postcode'].value_counts()

3046    91
3161    86
3338    58
3004    57
3060    43
3024    39
3195    35
3335    30
3155    23
3337    22
3064    22
3012    21
3977    19
3550    15
3016    14
3940    13
3037    11
3043     9
3803     9
3026     8
3030     8
3021     7
3551     7
3690     5
3556     4
3220     2
3260     2
3121     2
3350     2
3756     1
3805     1
3915     1
3950     1
3971     1
3185     1
3215     1
3207     1
3196     1
3181     1
3135     1
3128     1
3084     1
3072     1
3041     1
3022     1
3015     1
3995     1
Name: postcode, dtype: int64

In [12]:
# 3000 data is filled in 3004
domain.loc[domain['postcode'] == 3004, 'suburb_sold'] = '596'
domain.loc[domain['postcode'] == 3004, 'suburb_avg_day'] = '147'
domain.loc[domain['postcode'] == 3004, 'suburb_population'] = '6423'
domain.loc[domain['postcode'] == 3004, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3004, 'suburb_owner'] = '30%'
domain.loc[domain['postcode'] == 3004, 'suburb_renter'] = '70%'
domain.loc[domain['postcode'] == 3004, 'suburb_family'] = '24%'
domain.loc[domain['postcode'] == 3004, 'suburb_single'] = '76%'

In [13]:
domain.loc[domain['postcode'] == 3046, 'suburb_sold'] = '178'
domain.loc[domain['postcode'] == 3046, 'suburb_avg_day'] = '48'
domain.loc[domain['postcode'] == 3046, 'suburb_population'] = '22246'
domain.loc[domain['postcode'] == 3046, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3046, 'suburb_owner'] = '63%'
domain.loc[domain['postcode'] == 3046, 'suburb_renter'] = '37%'
domain.loc[domain['postcode'] == 3046, 'suburb_family'] = '49%'
domain.loc[domain['postcode'] == 3046, 'suburb_single'] = '51%'

In [14]:
domain.loc[domain['postcode'] == 3024, 'suburb_sold'] = '288'
domain.loc[domain['postcode'] == 3024, 'suburb_avg_day'] = '40'
domain.loc[domain['postcode'] == 3024, 'suburb_population'] = '23294'
domain.loc[domain['postcode'] == 3024, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3024, 'suburb_owner'] = '72%'
domain.loc[domain['postcode'] == 3024, 'suburb_renter'] = '28%'
domain.loc[domain['postcode'] == 3024, 'suburb_family'] = '51%'
domain.loc[domain['postcode'] == 3024, 'suburb_single'] = '49%'

In [15]:
domain.loc[domain['postcode'] == 3337, 'suburb_sold'] = '99'
domain.loc[domain['postcode'] == 3337, 'suburb_avg_day'] = '43'
domain.loc[domain['postcode'] == 3337, 'suburb_population'] = '17589'
domain.loc[domain['postcode'] == 3337, 'suburb_age'] = '20 to 39'
domain.loc[domain['postcode'] == 3337, 'suburb_owner'] = '74%'
domain.loc[domain['postcode'] == 3337, 'suburb_renter'] = '26%'
domain.loc[domain['postcode'] == 3337, 'suburb_family'] = '46%'
domain.loc[domain['postcode'] == 3337, 'suburb_single'] = '54%'

In [16]:
for post in domain.postcode:
    domain =  domain.drop(domain[(domain['postcode'] == post)&(domain['suburb_population'].isnull())].index)

In [17]:
# conver str typr data to numeric data
domain['Beds'] = domain['Beds'].str.extract(r'([0-9]+)')
domain['Bath'] = domain['Bath'].str.extract(r'([0-9]+)')
domain['Parking'] = domain['Parking'].str.extract(r'([0-9]+)')
domain['suburb_owner'] = domain['suburb_owner'].str.extract(r'([0-9]+)')
domain['suburb_renter'] = domain['suburb_renter'].str.extract(r'([0-9]+)')
domain['suburb_family'] = domain['suburb_family'].str.extract(r'([0-9]+)')
domain['suburb_single'] = domain['suburb_single'].str.extract(r'([0-9]+)')
domain['school_distance'] = domain['school_distance'].str.extract(r'([0-9]+.[0-9]*)')
domain['suburb_age'] = domain['suburb_age'].str.extract(r'([0-9]+)')
domain['suburb_population'] = domain['suburb_population'].str.replace(",", "")
domain['suburb_sold'] = domain['suburb_sold'].str.replace("-", "0")
domain['suburb_avg_day'] = domain['suburb_avg_day'].str.replace("-", "0")
domain[['Beds', 'Bath','Parking','suburb_population','suburb_owner','suburb_renter','suburb_avg_day',\
        'suburb_sold','suburb_family','suburb_single','school_distance','suburb_age']] =\
domain[['Beds', 'Bath','Parking','suburb_population','suburb_owner','suburb_renter','suburb_avg_day',\
        'suburb_sold','suburb_family','suburb_single','school_distance','suburb_age']].apply(pd.to_numeric)

In [39]:
domain.head()

,url,name,cost_text,type,school_title,school_distance,suburb_sold,suburb_avg_day,suburb_population,suburb_age,...,suburb_family,suburb_single,coordinates,rooms,desc,postcode,cost,Beds,Bath,Parking
0,https://www.domain.com.au/1202-620-collins-str...,1202/620 Collins Street Melbourne VIC 3000,$500 per week,Apartment / Unit / Flat,University High School,2.4,596.0,147.0,47279.0,20,...,24,76,"[-37.8186626, 144.9543471]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Ender Gok,3000,500.0,2.0,1.0,1.0
1,https://www.domain.com.au/1701-620-collins-str...,1701/620 Collins Street Melbourne VIC 3000,$650 per week,Apartment / Unit / Flat,University High School,2.4,596.0,147.0,47279.0,20,...,24,76,"[-37.8186626, 144.9543471]","[['2 Beds'], ['2 Baths'], ['1 Parking']]",Ender Gok,3000,650.0,2.0,2.0,1.0
2,https://www.domain.com.au/5707b-639-little-lon...,5707B/639 Little Lonsdale Street Melbourne VIC...,$570,Apartment / Unit / Flat,University High School,1.9,387.0,120.0,47279.0,20,...,24,76,"[-37.8141942, 144.9521662]","[['1 Bed'], ['1 Bath'], []]",Kim Te,3000,570.0,1.0,1.0,NaN
3,https://www.domain.com.au/1003-225-elizabeth-s...,1003/225 Elizabeth Street Melbourne VIC 3000,$575.00 per week,Apartment / Unit / Flat,Carlton Gardens Primary School,1.4,596.0,147.0,47279.0,20,...,24,76,"[-37.8132308, 144.9623173]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Barbara Baltas,3000,575.0,2.0,1.0,1.0
4,https://www.domain.com.au/1202-31-spring-stree...,1202/31 Spring Street Melbourne VIC 3000,$570.00,Apartment / Unit / Flat,Carlton Gardens Primary School,1.4,596.0,147.0,47279.0,20,...,24,76,"[-37.8145047, 144.9740695]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Bo Natoli,3000,570.0,2.0,1.0,1.0


### Locality to SA2 Coding Index
a csv saving convertation between SA2 and postcode

In [19]:
sa2_2016 = pd.read_csv("../data/raw/australian_postcodes.csv") # SA2 code in 2016
sa2_2021 = pd.read_csv("../data/raw/CG_SA2_2016_SA2_2021.csv")# SA2 code in 2021

In [20]:
# merge data and only keep SA2 in VIC
index = sa2_2021.merge(sa2_2016, on='SA2_MAINCODE_2016')
index = index[index['state'] == 'VIC']

In [21]:
# keep useful col and drop duplicates
index = index[['SA2_MAINCODE_2016', 'SA2_NAME_2016_x', 'SA2_CODE_2021', 'SA2_NAME_2021','postcode','locality','Lat_precise','Long_precise']]
index = index.drop_duplicates(subset=['postcode'])

In [22]:
postcode = pd.read_csv('../data/raw/Australian-Postcode-Data-master/au_postcodes.csv') # postcode in VIC
code_vic = postcode[postcode['state_code'] == 'VIC']
code_vic = code_vic[postcode['accuracy'] == 4] # only keep accuracy is 4 postcode that refer to a suburb
code_vic = code_vic.drop_duplicates(subset=['postcode'])

/var/folders/xx/831dtzvs55b0c08vg3m5yhf40000gn/T/ipykernel_14137/1310645104.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  code_vic = code_vic[postcode['accuracy'] == 4] # only keep accuracy is 4 postcode that refer to a suburb


In [23]:
index_vic = code_vic[['postcode','latitude','longitude']].merge(index, on='postcode')

In [24]:
index_vic.head()

,postcode,latitude,longitude,SA2_MAINCODE_2016,SA2_NAME_2016_x,SA2_CODE_2021,SA2_NAME_2021,locality,Lat_precise,Long_precise
0,3000,-37.8140,144.9633,206041122.0,Melbourne,206041503,Melbourne CBD - East,MELBOURNE,-37.815207,144.963937
1,3001,-37.8140,144.9633,206041122.0,Melbourne,206041503,Melbourne CBD - East,MELBOURNE,-37.813628,144.963058
2,3002,-37.8167,144.9879,206041119.0,East Melbourne,206041119,East Melbourne,EAST MELBOURNE,-37.816144,144.980459
3,3003,-37.8101,144.9500,206041127.0,West Melbourne,206041127,West Melbourne - Industrial,WEST MELBOURNE,-37.811450,144.925397
4,3004,-37.8140,144.9633,206041125.0,South Yarra - West,206041125,South Yarra - West,ST KILDA ROAD CENTRAL,-37.836800,144.976000


In [25]:
index_vic.to_csv('../data/curated/postcode_to_sa2.csv',index=False)

### Merge with the external data

In [26]:
df_2021 = pd.read_csv('../data/curated/2021_data.csv')
df_p2s = index_vic

In [27]:
df_p2s.rename(columns={'SA2_CODE_2021':'SA2_CODE21'}, inplace=True)
df_p2s[['SA2_CODE21']] = df_p2s[['SA2_CODE21']].apply(pd.to_numeric)

In [28]:
result = pd.merge(df_p2s[['SA2_CODE21','postcode']],df_2021, how='left', on=['SA2_CODE21'])

domain_final = pd.merge(domain, result[['SA2_CODE21','avg_income', 'Median_tot_fam_inc_weekly', 'Tot_P_P',
       'Median_age_persons', 'Median_rent_weekly', 'postcode']], how='left', on=['postcode'])

In [29]:
domain_final.to_csv('../data/curated/domain_2.csv',index=False)